## E91 Protocol Implementation (from QWorld @ Womanium 2023)

At the beginning of E91 protocol, Asja prepares EPR pairs:
$$\ket{\psi}=\frac{1}{\sqrt{2}}\ket{00} \pm \frac{1}{\sqrt{2}}\ket{11}$$

**Note:** There is no need to choose random bit sequence or encoding bases as we did in BB84. One of the properties of EPR pairs is a random outcome as a result of performed measurement.

For the measurements, our users use four bases: $Z$, $X$, $W$, $V$.

<b>Asja will choose between - $Z$, $X$, $W$.\
Balvis will choose between - $Z$, $W$, $V$.</b>

They need to choose measurement basis for each qubit independently and randomly.

<h3>Step 1: Distributing quantum states</h3>

<ul>
    <li>Asja prepares 200 EPR pairs.
    <li>Asja keeps one qubit of EPR pair and sends second to Balvis.
    <li>Asja randomly chooses measurement basis ($Z$,$X$ or $W$) and measures first qubit (qreg[0]).
    <li>Asja saves information about measurement basis that she used for each qubit.
    <li>Balvis randomly chooses measurement basis ($Z$, $W$ or $V$) and measures second qubit (qreg[1]).
    <li>Balvis saves information about measurement basis that he used for each qubit.

In [1]:
# import all necessary objects and methods for quantum circuits
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from random import randrange

In [10]:
#Registers for measurement bases and keys
basesAsja, keyAsja = [],[]
basesBalvis, keyBalvis = [],[]

for i in range(200):  #Asja prepares 200 EPR pairs
    qreg = QuantumRegister(2)
    creg = ClassicalRegister(2) 
    mycircuit = QuantumCircuit(qreg, creg)

    #Creating entanglement
    mycircuit.h(qreg[0])
    mycircuit.cx(qreg[0], qreg[1])

    #Asja chooses measurement basis
    choiceAsja=randrange(3)
    if choiceAsja==0:#measurement in Z basis
        basesAsja.append('Z')
    if choiceAsja==1:#measurement in X basis
        mycircuit.h(qreg[0])
        basesAsja.append('X')
    if choiceAsja==2:#measurement in W basis
        mycircuit.s(qreg[0])
        mycircuit.h(qreg[0])
        mycircuit.t(qreg[0])
        mycircuit.h(qreg[0])
        basesAsja.append('W')

    #Balvis chooses measurement basis
    choiceBalvis=randrange(3)
    if choiceBalvis==0:#measurement in Z basis
        basesBalvis.append('Z')
    if choiceBalvis==1:#measurement in W basis
        mycircuit.s(qreg[1])
        mycircuit.h(qreg[1])
        mycircuit.t(qreg[1])
        mycircuit.h(qreg[1])
        basesBalvis.append('W')
    if choiceBalvis==2:#measurement in V basis
        mycircuit.s(qreg[1])
        mycircuit.h(qreg[1])
        mycircuit.tdg(qreg[1])
        mycircuit.h(qreg[1])
        basesBalvis.append('V')

    mycircuit.measure(qreg,creg) #applying final measurement
    job = execute(mycircuit,Aer.get_backend('qasm_simulator'),shots=1)
    counts = job.result().get_counts(mycircuit)
    
    #Saving results
    result=list(counts.keys())[0] #retrieve key from dictionary
    keyAsja.append(int(result[0])) #saving first qubit value in Asja's key register 
    keyBalvis.append(int(result[1])) #and second to Balvis

<h3>Step 2: Sifting</h3>

Let's implement sifting phase.\
Asja and Balvis compare bases.\
If they used same measurement bases, they save corresponding bits to their final key strings.\
If bases mismatch, they keep corresponding bits and information about basis they used in separate registers.

In [11]:
#Registers
finalKeyAsja, finalKeyBalvis = [],[] #for matching rounds
diffAsja, diffBalvis = [],[] #missmatched rounds
diffBasesA, diffBasesB = [],[] #bases of missmatched rounds
for i in range(0, len(basesAsja)):
    if basesAsja[i] == basesBalvis[i]: #When used same bases
        finalKeyAsja.append(keyAsja[i])
        finalKeyBalvis.append(keyBalvis[i]) 
    else: #When used different
        diffAsja.append(keyAsja[i])
        diffBalvis.append(keyBalvis[i])
        diffBasesA.append(basesAsja[i])
        diffBasesB.append(basesBalvis[i])

<h3>QBER - CHSH Violation Test</h3>

When users measure qubits using same bases, the outcome is perfect correlation.
When they use different bases, the outcomes are completely random (uncorrelated).

To detect eavesdropping, Asja and Balvis can compute the test statistic $S$ using the correlation coefficients between Asja's bases and Balvis':
$$S=ZW+XW-XV+ZV$$

From previous notebook, we know that to calculate $S$ we will need correlation values for four cases:
<center> $ZW$ - Asja used $Z$ basis, Balvis used $W$
<center> $XW$ - Asja used $X$ basis, Balvis used $W$
<center> $XV$ - Asja used $X$ basis, Balvis used $V$
<center> $ZV$ - Asja used $Z$ basis, Balvis used $V$

For each case, correlation value can be calculated using formula:
<center>$⟨A\otimes B⟩=P(1,1)+P(0,0)-P(1,0)-P(0,1)$</center>

Here, $P(1,1)$ and $P(0,0)$ are probabilities: numbers of rounds where Asja and Balvis obtained identical bits after measurement ($11$ and $00$) divided by total number of rounds where measurement is done with these two bases.\
$P(1,0)$ and $P(0,1)$ are probabilities: numbers of rounds where Asja and Balvis obtained different bits ($10$ and $01$) divided by total number of rounds where measurement is done with these two bases.

<h3>Step 3: CHSH inequality violation test</h3>

Calculate correlation values and $S$.

In [12]:
#ZW
sameZW = 0
diffZW = 0

for i, (bA, bB) in enumerate(zip(diffBasesA, diffBasesB)):
    if (bA == 'Z' and bB == 'W'):
        if diffAsja[i]==diffBalvis[i]:
            sameZW=sameZW+1
        else:
            diffZW=diffZW+1

totalZW=sameZW+diffZW
if totalZW!=0:
    ZW=(sameZW-diffZW)/totalZW
else:
    ZW=0

In [13]:
#XW
sameXW = 0
diffXW = 0
for i, (bA, bB) in enumerate(zip(diffBasesA, diffBasesB)):
    if (bA == 'X' and bB == 'W'):
        if diffAsja[i]==diffBalvis[i]:
            sameXW=sameXW+1
        else:
            diffXW=diffXW+1

totalXW=sameXW+diffXW
if totalXW!=0:
    XW=(sameXW-diffXW)/totalXW
else:
    XW=0

In [14]:
#XV
sameXV = 0
diffXV = 0
for i, (bA, bB) in enumerate(zip(diffBasesA, diffBasesB)):
    if (bA == 'X' and bB == 'V'):
        if diffAsja[i]==diffBalvis[i]:
            sameXV=sameXV+1
        else:
            diffXV=diffXV+1

totalXV=sameXV+diffXV
if totalXV!=0:
    XV=(sameXV-diffXV)/totalXV
else:
    XV=0

In [15]:
#ZV
sameZV = 0
diffZV = 0
for i, (bA, bB) in enumerate(zip(diffBasesA, diffBasesB)):
    if (bA == 'Z' and bB == 'V'):
        if diffAsja[i]==diffBalvis[i]:
            sameZV=sameZV+1
        else:
            diffZV=diffZV+1

totalZV=sameZV+diffZV
if totalZV!=0:
    ZV=(sameZV-diffZV)/totalZV

else:
    ZV=0

In [16]:
S=ZW+XW-XV+ZV

print("CHSH inequality value is", S)

CHSH inequality value is 2.7698412698412698


Maximally entangled photons would result in $S= ± 2\sqrt{2}$.

If this were not the case, then Asja and Balvis can conclude Espian has introduced errors to the system. 

If $S$ 
value corresponds to quantum system, Asja and Balvis can use bits from matching round as a secret key!

In [17]:
print(finalKeyAsja)
print(finalKeyBalvis)

[1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0]


<font style="color:darkorange;font-weight:bold;"> <h2>Task 1</h2> </font>

Re-run this program for different number of EPR pairs.

Hint: You only need to change the number of loop iterations at the very start.

Next: [Security of E91 Protocol](QC22_Security_of_E91_protocol.ipynb)